In [ ]:
import skimage # sibling of sklearn but for Image PRocessing
import numpy as np # we all know
import pandas as pd # evolved numpy
import matplotlib.pyplot as plt # This one is a true creator
import warnings # No one liked warnings
from keras.datasets import cifar10 # image data
import random # to perform random tasks

In [ ]:
warnings.filterwarnings('ignore') # ignore all warning because I don't care what they say, I gotta learn

# Basic Operations

## Reading and Plotting Image

In [ ]:
chelsea = skimage.data.chelsea() # data is given here already
print(chelsea.shape)
plt.imshow(chelsea)
plt.show()

## Flip and Rotate

In [ ]:
plt.imshow(np.fliplr(chelsea))

In [ ]:
plt.imshow(skimage.transform.rotate(chelsea,90))

In [ ]:
plt.imshow(np.rot90(chelsea))

## Masking

In [ ]:
blue_chelsea = chelsea.copy()
mask = (blue_chelsea[:,:,0]>103) & (blue_chelsea[:,:,0]<153)  # get random pixels based on a condition
blue_chelsea[mask] = [0,0,255] # turn the mask as blue
plt.imshow(blue_chelsea)

## Color scheme Conversion

### RGB to Grayscale
3 channels merged into one. 0-255 range becomes 0 or 1

In [ ]:
plt.imshow(skimage.color.rgb2gray(chelsea),cmap='gray') # RGB to grayscale

### Invert colors
High intensity pixels are interchanges with low intensity pixels

In [ ]:
plt.imshow(skimage.util.invert(chelsea)) # inverted colors

### RGB to BGR

In [ ]:
BGR_chelsea = chelsea[:,:,::-1]
plt.imshow(BGR_chelsea) # RGB to BGR

## Crop

In [ ]:
def crop_center(img,y,x):
    h,w,c = img.shape
    start_w = w//2 - (x//2)
    start_h = h//2 - (y//2)
    return img[start_h:start_h+y,start_w:start_w+w]

In [ ]:
plt.imshow(crop_center(chelsea,300,200))

# Pooling
A lying or standing cat is still a cat due to its features not the location of features. Convolution layers sometimes give importance to the location so pooling tries to minimize that effect.
[Pooling Layers](https://iq.opengenus.org/pooling-layers/)

In [ ]:
gray = skimage.color.rgb2gray(chelsea)
resized = skimage.transform.resize(gray,(300,448)) # original is 300,451

In [ ]:
kernal = (4,4) # or block
blocked = skimage.util.view_as_blocks(resized,kernal)
# w,h should be completely divided by kernal size
blocked.shape

In [ ]:
# reshape the image while preserving the Width and Height OR simply merge color and patches channels
reshaped = blocked.reshape(blocked.shape[0],blocked.shape[1],-1)
reshaped.shape

In [ ]:
# MAx pooling
max_pooled = np.max(reshaped,axis=2)
avg_pooled = np.mean(reshaped,axis=2)
min_pooled = np.min(reshaped,axis=2)
med_pooled = np.median(reshaped,axis=2)

f,ax = plt.subplots(2,2,figsize=(15,8))
ax = ax.ravel()

ax[0].imshow(max_pooled,cmap='gray')
ax[0].set_title('Max Pooled')
ax[0].axis('off')

ax[1].imshow(min_pooled,cmap='gray')
ax[1].set_title('Min Pooled')
ax[1].axis('off')

ax[2].imshow(avg_pooled,cmap='gray')
ax[2].set_title('Average Pooled')
ax[2].axis('off')

ax[3].imshow(med_pooled,cmap='gray')
ax[3].set_title('Median Pooled')
ax[3].axis('off')


plt.show()

# ZCA Whitening and Normalization

In [ ]:
(_,_),(images,_) = cifar10.load_data() 
# (X_train,y_tain),(x_test,y_test) but we need less in quantity so we just imported x_test

images = images[:1000] # just use the first 1000 images for demo

print(f'Shape of images array is: {images.shape}')

f,ax = plt.subplots(1,3,figsize=(5,5))
ax = ax.ravel()
for i in range(3):
    ax[i].imshow(images[i])
plt.show()

In [ ]:
images = images.reshape(-1,(32*32*3)) # -1 means it choses the best suitable i.e 1000 in our case
print(f' new shape of images is {images.shape}')

## ZCA Whitening
ZCA decorrelates the Image Features or transforms the image in such a way that makes the covariance matrix of the image as identity matrix

In [ ]:
 # find the covariance matrix. i.e how much two variables change together
cov_mat = np.cov(images)

# singular value decomposition SVD (dimensionality reduction technique to find  hidden latent features)
# a plastic bag, douchebag and a weapon can have a latent feature that they are all from USA ;)
U,S,V = np.linalg.svd(cov_mat)

# dot product to get the principal components
epsilon = 0.000001 # avoid division by zero
w = np.diag(1.0/np.sqrt(S+epsilon)) # diagonal matrix
x = np.dot(w,U.T) # U transpose     
components = np.dot(U,x)

# calculate zca by using dot products of the principal components by images 
zca_images = np.dot(components,images)
zca_images.shape

In [ ]:
f,ax = plt.subplots(1,3,figsize=(5,5))
ax = ax.ravel()
for i in range(3):
    # clip the images in some range for matplotlib else it'll throw error
    img = zca_images[i].reshape((32,32,3))
    min_,max_ = img.min(), img.max()
    ax[i].imshow((img-min_)/(max_-min_))  # clipping
plt.show()

## Normalization
Normalization makes the pixel centered of image around mean 0 by subtracting the pixels of an image from the mean of whole **batch** of images and dividing by the standard deviation of the **batch**

In [ ]:
images = images - images.mean(axis=0) # subtract the mean of whole 10000 images from each image
images = images/images.std(axis=0) # divide by the standard deviation

f,ax = plt.subplots(1,3,figsize=(5,5))
ax = ax.ravel()
for i in range(3):
    # clip the images in some range for matplotlib else it'll throw error
    img = images[i].reshape((32,32,3))
    min_,max_ = img.min(), img.max()
    ax[i].imshow((img-min_)/(max_-min_))  # clipping
plt.show()

# De-Noising
To save time here, let us just resize the image then we will add Gaussian Noise. 

In [ ]:
resized = skimage.transform.resize(chelsea,(chelsea.shape[0]//2,chelsea.shape[1]//2)) 
# resize to half preserving the aspect ratio
plt.imshow(resized)

In [ ]:
# add Gaussian Noise
sigma = 0.17 # defines the type/shape of distribution high sigma = high noise
noised = skimage.util.random_noise(resized,mode='gaussian', var=sigma**2)
plt.imshow(noised)

In [ ]:
# if our image has Gaussian noise, this can detect and tell us the sigma. Our result is close to our sigma

skimage.restoration.estimate_sigma(noised,multichannel=True,average_sigmas=True)
# we want to calculate the noise for each of the RGB channel (multichannel=True) and want avg. for all

### TV Chambolle

In [ ]:
# it makes the 'normalization' value close to the normal image by using 100 iteration by default

tv_cham = skimage.restoration.denoise_tv_chambolle(noised,multichannel=True,weight=0.12)
plt.imshow(tv_cham)
# output depends on the weight. High value of weight makes the image blurry and far from original

### Bilateral

In [ ]:
# this preserves the edges and works on closeness of pixels (spatial closeness) and 
# how two pixels are similar in their color channels (radiometric similarity)

bilat = skimage.restoration.denoise_bilateral(noised,multichannel=True,sigma_color=0.09,sigma_spatial=1.3)
plt.imshow(bilat)
# both the sigma as parameters are standard deviations for spatial closeness and radiometric similarity
# try to use high sigma_spatial. It'll take lot of time and the black portion at the bottom will increase

### Wavelet

In [ ]:
# this works on the wavelength representation of image and follows the luminosity (Y) and chroma components
# (Cb,Cr) so it is YCbCr instead of RGB format

wave = skimage.restoration.denoise_wavelet(noised,multichannel=True,wavelet='db2',method='VisuShrink')
plt.imshow(wave)

# try to read the documentation about the parameters. These vary from image to image

### Non Local Means

In [ ]:
# it takes a mean of all pixels, weighted by how similar these pixels are to the target pixel. 
# This results in greater post-filtering clarity than local means
nl_mean = skimage.restoration.denoise_nl_means(noised,multichannel=True,patch_size=6,patch_distance=13)
plt.imshow(nl_mean)

In [ ]:
! git init
! git add -A
! git commit -m "ZCA "